In [ ]:
import pandas as pd
import os
import joblib
import librosa
import numpy as np

In [ ]:
df_2021 = pd.read_csv("../inputs/pretrain/birdclef-2021/train_metadata.csv")
df_2021["path"] = "../inputs/pretrain/birdclef-2021/train_short_audio/" + df_2021["primary_label"] + "/" + df_2021["filename"]

In [ ]:
df_2022 = pd.read_csv("../inputs/pretrain/birdclef-2022/train_metadata.csv")
df_2022["path"] = "../inputs/pretrain/birdclef-2022/train_audio/" + df_2022["filename"]

In [ ]:
df_2023 = pd.read_csv("../inputs/pretrain/birdclef-2023/train_metadata.csv")
df_2023["path"] = "../inputs/pretrain/birdclef-2023/train_audio/" + df_2023["filename"]

In [ ]:
# joblib.Parallel(n_jobs=10, verbose=5)
def get_audio_meta(path):
    x, sr = librosa.load(path, mono=True)
    return len(x), sr, np.abs(x.max())

meta_2021 = joblib.Parallel(n_jobs=10, verbose=5)(joblib.delayed(get_audio_meta)(p) for p in df_2021["path"])

In [ ]:
meta_2022 = joblib.Parallel(n_jobs=10, verbose=5)(joblib.delayed(get_audio_meta)(p) for p in df_2022["path"])

In [ ]:
meta_2023 = joblib.Parallel(n_jobs=10, verbose=5)(joblib.delayed(get_audio_meta)(p) for p in df_2023["path"])


In [ ]:
def merge_metadata(meta_pairs, base_df):
    df_meta = pd.DataFrame(meta_pairs, columns=["duration", "sr"])
    df_meta["duration_sec"] = df_meta["duration"] / df_meta["sr"]

    new_df = pd.concat([base_df, df_meta], axis=1)
    return new_df

In [ ]:
df_2021_rich = merge_metadata(meta_2021, df_2021)
df_2021_rich.to_parquet("train_metadata_rich_2022.parquet")

In [ ]:
df_2022_rich = merge_metadata(meta_2022, df_2022)
df_2022_rich.to_parquet("train_metadata_rich_2022.parquet")

In [ ]:
df_2023_rich = merge_metadata(meta_2023, df_2023)
df_2023_rich.to_parquet("train_metadata_rich_2023.parquet")

## Merge 2021-2023 data

In [ ]:
# prepare dataframe
def load_df():
    df_2021 = pd.read_parquet("train_metadata_rich_2021.parquet")
    df_2022 = pd.read_parquet("train_metadata_rich_2022.parquet")
    df_2023 = pd.read_parquet("train_metadata_rich_2023.parquet")
    return pd.concat([df_2021, df_2022, df_2023]).reset_index(drop=True)

df = load_df()

primary_label_count = df.primary_label.value_counts()
sample_weights = (
    primary_label_count / 
    primary_label_count.sum()
)  ** (-0.5)
df["weight"] = sample_weights[df["primary_label"].values].values
df["weight"] = (df["weight"] / df["weight"].mean())

In [ ]:
# check exist files
assert df["path"].apply(os.path.exists).all()
df["path"] = df["path"].apply(os.path.abspath)

# save merged df
pretrain_filename = "train_metadata_rich_pretrain_merge.pkl"
df.to_pickle(pretrain_filename)